Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [2]:
from itertools import permutations
from tqdm.auto import tqdm
from icecream import ic

In [3]:
DIM = 2
GOAL_STATE = tuple(list(range(1, DIM**2)) + [0])

In [4]:
def print_board(board):
    r"""Print board"""
    print("\n".join("|".join(f'{i:1}' for i in board[r * DIM : (r + 1) * DIM]) for r in range(DIM)))


print_board(GOAL_STATE)


def i2rc(i):
    r"""Convert index to (row, column)"""
    return divmod(i, DIM)


def rc2i(r, c):
    r"""Convert (row, column) to index"""
    return r * DIM + c

1|2
3|0


In [5]:
ACTIONS = list()
for i in range(DIM**2):
    valid = list()
    r, c = i2rc(i)
    if r > 0:
        valid.append(rc2i(r - 1, c))
    if r < DIM - 1:
        valid.append(rc2i(r + 1, c))
    if c > 0:
        valid.append(rc2i(r, c - 1))
    if c < DIM - 1:
        valid.append(rc2i(r, c + 1))
    ACTIONS.append(tuple(valid))

for i, a in enumerate(ACTIONS):
    actions = [i2rc(_) for _ in a]
    state = i2rc(i)
    ic(state, actions)

ic| state:

 (0, 0), actions: [(1, 0), (0, 1)]
ic| state: (0, 1), actions: [(1, 1), (0, 0)]
ic| state: (1, 0), actions: [(0, 0), (1, 1)]
ic| state: (1, 1), actions: [(0, 1), (1, 0)]


In [6]:
def do_action(state, action):
    r"""Perform action in state, returns new state"""
    new_state = list(state)
    new_state[state.index(0)] = new_state[action]
    new_state[action] = 0
    return tuple(new_state)


def get_possible_actions(state):
    r"""Gets a list of sction/reward for current state"""
    if state == GOAL_STATE:
        return [(state.index(0), 0)]
    else:
        return [(a, -1) for a in ACTIONS[state.index(0)]]

In [ ]:
def greedy_policy(state, value):
    r"""Given a state a a value function, returns list of optimal greedy moves with associated immediate reward"""
    q = dict()
    for a, r in get_possible_actions(state):
        new_state = do_action(state, a)
        q[a] = r + value[new_state]
    max_v = max(q.values())
    return set((a, -1 if v < 0 else 0) for a, v in q.items() if v == max_v)


def random_policy(state):
    r"""Given a state a a value function, returns list of optimal greedy moves with associated immediate reward"""
    return set(get_possible_actions(state))


def describe(value):
    for i in range(1, 1 - int(min(value.values()))):
        s = [s for s, v in value.items() if v == -i]
        if s:
            print(f"Found {len(s):,} states at distance {i}")

## Value Iteration

In [20]:
value = {tuple(s): 0 for s in permutations(range(DIM**2))}
print(f"v() contains {len(value):,} states")

current_policy = {s: random_policy(s) for s in value.keys()}
for steps in tqdm(range(1000)):
    new_value = dict()
    for state in value:
        new_value[state] = 0
        actions = current_policy[state]
        for a, r in actions:
            new_value[state] += 1 / len(actions) * (r + value[do_action(state, a)])
    value = new_value

value

v() contains 24 states


  0%|          | 0/1000 [00:00<?, ?it/s]

{(0, 1, 2, 3): -1000.0,
 (0, 1, 3, 2): -19.99999999999997,
 (0, 2, 1, 3): -19.99999999999997,
 (0, 2, 3, 1): -1000.0,
 (0, 3, 1, 2): -1000.0,
 (0, 3, 2, 1): -35.99999999999994,
 (1, 0, 2, 3): -1000.0,
 (1, 0, 3, 2): -10.999999999999986,
 (1, 2, 0, 3): -10.999999999999986,
 (1, 2, 3, 0): 0.0,
 (1, 3, 0, 2): -1000.0,
 (1, 3, 2, 0): -1000.0,
 (2, 0, 1, 3): -26.99999999999996,
 (2, 0, 3, 1): -1000.0,
 (2, 1, 0, 3): -1000.0,
 (2, 1, 3, 0): -1000.0,
 (2, 3, 0, 1): -34.99999999999994,
 (2, 3, 1, 0): -31.99999999999995,
 (3, 0, 1, 2): -1000.0,
 (3, 0, 2, 1): -34.99999999999994,
 (3, 1, 0, 2): -26.99999999999996,
 (3, 1, 2, 0): -31.99999999999995,
 (3, 2, 0, 1): -1000.0,
 (3, 2, 1, 0): -1000.0}

In [16]:
value = {tuple(s): 0 for s in permutations(range(DIM**2))}
print(f"v() contains {len(value):,} states")

current_policy = dict()
new_policy = {s: random_policy(s) for s in value.keys()}

stopping_condition = False
steps = 0
with tqdm() as pbar:
    while not stopping_condition:
        steps += 1
        current_policy = new_policy
        new_value = dict()
        for state in value:
            new_value[state] = 0
            actions = current_policy[state]
            for a, r in actions:
                new_value[state] += 1 / len(actions) * (r + value[do_action(state, a)])
        value = new_value
        new_policy = {s: greedy_policy(s, value) for s in value.keys()}
        pbar.update(1)
        if steps == 30:
            stopping_condition = True
ic(value)
None

v() contains 24 states


0it [00:00, ?it/s]

ic| value: {(0, 1, 2, 3): -30.0,
            (0, 1, 3, 2): -2.0,
            (0, 2, 1, 3): -2.0,
            (0, 2, 3, 1): -30.0,
            (0, 3, 1, 2): -30.0,
            (0, 3, 2, 1): -6.0,
            (1, 0, 2, 3): -30.0,
            (1, 0, 3, 2): -1.0,
            (1, 2, 0, 3): -1.0,
            (1, 2, 3, 0): 0.0,
            (1, 3, 0, 2): -30.0,
            (1, 3, 2, 0): -30.0,
            (2, 0, 1, 3): -3.0,
            (2, 0, 3, 1): -30.0,
            (2, 1, 0, 3): -30.0,
            (2, 1, 3, 0): -30.0,
            (2, 3, 0, 1): -5.0,
            (2, 3, 1, 0): -4.0,
            (3, 0, 1, 2): -30.0,
            (3, 0, 2, 1): -5.0,
            (3, 1, 0, 2): -3.0,
            (3, 1, 2, 0): -4.0,
            (3, 2, 0, 1): -30.0,
            (3, 2, 1, 0): -30.0}


Found 12 states at distance 29
Found 12 states at distance 30


In [ ]:
for s, v in value.items():
    if v == -2:
        print_board(s)
        print(greedy_policy(s, value))
        print()